In [1]:
!pip install tensorflow==2.11

ERROR: Could not find a version that satisfies the requirement tensorflow==2.11 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.11


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from tqdm import tqdm
# Import VGG16 from tensorflow.keras.applications
from tensorflow.keras.applications import vgg16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
# Import one_hot from tensorflow.keras.preprocessing.text
from tensorflow.keras.preprocessing.text import one_hot
# Import pad_sequences from tensorflow.keras.preprocessing.sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from nltk.translate.bleu_score import sentence_bleu
from numpy import array
import pickle
import numpy as np
import os
import cv2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/advanced-ml-second-assignment-2025 (1).zip'
 # replace with your actual file path
extract_to = '/content/data'  # where you want to extract

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [5]:
base_path = '/content/data'
train_img_path = os.path.join(base_path, 'train', 'train')
val_img_path = os.path.join(base_path, 'val', 'val')
test_img_path = os.path.join(base_path, 'test', 'test')
train_txt = os.path.join(base_path, 'train.txt')
val_txt = os.path.join(base_path, 'val.txt')


In [6]:
def load_captions(filepath):
    captions = {}
    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if '\t' in line:
                img_id, caption = line.split('\t')
                img_id = img_id.split('#')[0]  # Remove #0, #1 etc.
                captions.setdefault(img_id, []).append(caption)
    return captions

train_captions_dict = load_captions('/content/data/train.txt')
val_captions_dict = load_captions('/content/data/val.txt')

print("Sample train captions:")
for img_id, caps in list(train_captions_dict.items())[:2]:
    print(img_id, "→", caps)


Sample train captions:
2874876837_80d178ba9b.jpg → ['A boy bouncing on a trampoline .', 'A boy jumping on a trampoline .', 'A boy jumps on a trampoline .', 'A little boy in a white long sleeved t-shirt jumps on a trampoline .', 'Boy jumps on trampoline outside house .']
636503038_17ca82b50f.jpg → ['"Three fishers walk over rocks toward water , lake and cloudy sky in background ."', 'Three people are standing on rocks near a body of water .', 'Three people are walking on rocks towards a lake .', 'Three people stand with fishing poles on the rocks near the water .', '"Three people walking along rocks , going to go fish"']


In [7]:
import string

def clean_caption(caption):
    caption = caption.lower()
    caption = caption.translate(str.maketrans('', '', string.punctuation))
    caption = ' '.join([word for word in caption.split() if word.isalpha()])
    caption = '<start> ' + caption + ' <end>'
    return caption

# Clean train captions
cleaned_train_captions_dict = {
    img_id: [clean_caption(c) for c in captions]
    for img_id, captions in train_captions_dict.items()
}

# Clean val captions (optional now, but useful later for validation)
cleaned_val_captions_dict = {
    img_id: [clean_caption(c) for c in captions]
    for img_id, captions in val_captions_dict.items()
}

# Print a few samples from the cleaned training captions
print("\n📷 Sample Cleaned Training Captions:")
for img_id, captions in list(cleaned_train_captions_dict.items())[:2]:
    print(f"{img_id} →")
    for cap in captions:
        print(f"  {cap}")



📷 Sample Cleaned Training Captions:
2874876837_80d178ba9b.jpg →
  <start> a boy bouncing on a trampoline <end>
  <start> a boy jumping on a trampoline <end>
  <start> a boy jumps on a trampoline <end>
  <start> a little boy in a white long sleeved tshirt jumps on a trampoline <end>
  <start> boy jumps on trampoline outside house <end>
636503038_17ca82b50f.jpg →
  <start> three fishers walk over rocks toward water lake and cloudy sky in background <end>
  <start> three people are standing on rocks near a body of water <end>
  <start> three people are walking on rocks towards a lake <end>
  <start> three people stand with fishing poles on the rocks near the water <end>
  <start> three people walking along rocks going to go fish <end>


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Combine all cleaned captions into a single list
all_captions = []
for captions in cleaned_train_captions_dict.values():
    all_captions.extend(captions)

for captions in cleaned_val_captions_dict.values():
    all_captions.extend(captions)

# Initialize and fit tokenizer
tokenizer = Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(all_captions)

# Save tokenizer for later use
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

train_sequences = tokenizer.texts_to_sequences(all_captions)
max_caption_length = 37  # You can adjust this as needed

# Pad the sequences so that all sequences are of the same length
train_seqs = pad_sequences(train_sequences, maxlen=max_caption_length, padding='post')


# Verify the shape of the padded sequences
print(f"Shape of train captions: {train_seqs.shape}")


# Vocabulary size (+1 for padding)
vocab_size = len(tokenizer.word_index) + 1

# Example stats
print(f"Vocabulary size: {vocab_size}")
print("Sample word index:", {k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]})


Shape of train captions: (36406, 37)
Vocabulary size: 8398
Sample word index: {'<unk>': 1, 'a': 2, 'end': 3, 'start': 4, 'in': 5, 'the': 6, 'on': 7, 'is': 8, 'and': 9, 'dog': 10}


In [ ]:
print("Sample sequences after tokenization and padding:")
for seq in train_seqs[:2]:
    print(seq)

Sample sequences after tokenization and padding:
[   4    2   17 1094    7    2  388    3    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]
[  4   2  17  45   7   2 388   3   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]


In [ ]:
model=ResNet50(include_top=False, weights='imagenet',pooling='avg',input_shape=(224,224,3))
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
#extracting image encodings(features) from resnet50 and forming dict train_features
path=train_img_path
train_features={}
c=0
for image_name in tqdm(train_captions_dict):
  img_path = os.path.join(path, image_name)
  if not os.path.isfile(img_path):
        continue
  img = image.load_img(img_path, target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  train_features[image_name]=features.squeeze()

with open( "train_encoded_images.p", "wb" ) as pickle_f:
   pickle.dump(train_features, pickle_f )

  0%|          | 0/6473 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


  0%|          | 1/6473 [00:00<13:12,  8.16it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


  0%|          | 2/6473 [00:00<12:30,  8.62it/s]

...

100%|█████████▉| 6471/6473 [14:42<00:00,  6.82it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


100%|█████████▉| 6472/6473 [14:43<00:00,  6.94it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


100%|██████████| 6473/6473 [14:43<00:00,  7.33it/s]


In [ ]:
#extracting image encodings(features) from resnet50 and forming dict test_features
path=test_img_path
test_features={}
c=0
img_dir=path
image_files = [img for img in os.listdir(img_dir) if img.lower().endswith('.jpg')]
for index, image_name in enumerate(image_files):
  img_path = os.path.join(img_dir, image_name)
  img=image.load_img(img_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  test_features[image_name]=features.squeeze()

  with open( "test_encoded_images.p", "wb" ) as pickle_f:
    pickle.dump(test_features, pickle_f )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━

In [ ]:
path=val_img_path
validation_features={}
c=0
for image_name in tqdm(val_captions_dict):
  img_path = os.path.join(path, image_name)
  img=image.load_img(img_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  features = model.predict(x)
  validation_features[image_name]=features.squeeze()

with open( "validation_encoded_images.p", "wb" ) as pickle_f:
  pickle.dump(validation_features, pickle_f )

  0%|          | 0/809 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


  0%|          | 1/809 [00:05<1:09:37,  5.17s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step


  0%|          | 2/809 [00:05<31:36,  2.35s/it]  

...

100%|█████████▉| 807/809 [05:25<00:00,  2.66it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step


100%|█████████▉| 808/809 [05:25<00:00,  2.82it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step


100%|██████████| 809/809 [05:26<00:00,  2.48it/s]


In [ ]:
all_words=" ".join(all_captions)
print()
print(len(all_words))
unique_words=list(set(all_words.strip().split(" ")))
print(len(unique_words))

#forming dictionaries containg mapping of words to indices and indices to words
words_to_indices={val:index+1 for index, val in enumerate(unique_words)}
indices_to_words = { index+1:val for index, val in enumerate(unique_words)}
words_to_indices["Unk"]=0
indices_to_words[0]="Unk"


2479150
8398


In [9]:
# Load the pickled train_features data
with open("/content/train_encoded_images.p", "rb") as pickle_f:
    train_features = pickle.load(pickle_f)

with open("/content/test_encoded_images.p", "rb") as pickle_f:
    test_features = pickle.load(pickle_f)


with open("/content/validation_encoded_images.p", "rb") as pickle_f:
    validation_features = pickle.load(pickle_f)



In [40]:
print(list(train_features.items())[:5])

[('2874876837_80d178ba9b.jpg', array([0.46016395, 1.9588624 , 0.        , ..., 0.08110321, 0.03742253,
       0.50726694], dtype=float32)), ('636503038_17ca82b50f.jpg', array([0.97925794, 0.06608035, 1.0114158 , ..., 0.4754986 , 0.36413157,
       0.38506743], dtype=float32)), ('3163068926_d28ed3ef53.jpg', array([1.1501114 , 0.44403845, 0.5529145 , ..., 1.6089246 , 2.3605936 ,
       0.16654162], dtype=float32)), ('2599131872_65789d86d5.jpg', array([0.36433932, 0.06762137, 0.01645433, ..., 0.6871026 , 0.3591541 ,
       0.4728736 ], dtype=float32)), ('2343879696_59a82f496f.jpg', array([0.22649437, 0.20236148, 0.3908168 , ..., 0.23715486, 0.05413714,
       0.22296582], dtype=float32))]


In [10]:
train_encoded_captions = {}
for img_id, captions in cleaned_train_captions_dict.items():
    train_encoded_captions[img_id] = tokenizer.texts_to_sequences(captions)

In [11]:
val_encoded_captions = {}
for img_id, captions in cleaned_val_captions_dict.items():
    train_encoded_captions[img_id] = tokenizer.texts_to_sequences(captions)

In [12]:
def data_generator(train_encoded_captions,train_features,num_of_photos):
  X1, X2, Y = list(), list(), list()
  max_length=37
  n=0
  for img_id in tqdm(train_encoded_captions,disable=True):
    if img_id not in train_features:
            continue
    n+=1
    for i in range(5):
      for j in range(1,len(train_encoded_captions[img_id][i])):
        curr_sequence=train_encoded_captions[img_id][i][0:j]
        next_word=train_encoded_captions[img_id][i][j]
        curr_sequence=pad_sequences([curr_sequence], maxlen=max_length, padding='post')[0]
        one_hot_next_word=to_categorical([next_word],vocab_size)[0]
        X1.append(train_features[img_id].reshape((2048,)))
        X2.append(curr_sequence)
        Y.append(one_hot_next_word)
    if(n==num_of_photos):
      X1 = np.array(X1).reshape(-1, 2048)  # Reshape X1 to (batch_size, 2048)
      X2 = np.array(X2).reshape(-1, max_length)
      yield ((array(X1), array(X2)), array(Y))
      X1, X2, Y = list(), list(), list()
      n=0
  if n > 0:
      X1 = np.array(X1).reshape(-1, 2048)  # Reshape X1 to (batch_size, 2048)
      X2 = np.array(X2).reshape(-1, max_length)
      yield ((array(X1), array(X2)), array(Y))

In [41]:
# Test one iteration of the generator
gen = data_generator(train_encoded_captions, train_features, num_of_photos=5)

# Get one batch from the generator
X, Y = next(gen)

# Print the shapes of X1, X2, and Y
print(f"X1 shape: {X[0].shape}, X2 shape: {X[1].shape}, Y shape: {Y.shape}")

X1 shape: (270, 2048), X2 shape: (270, 37), Y shape: (270, 8398)


In [13]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM  # Corrected import
from keras.layers import add
from keras.layers import Embedding


In [15]:
for i in train_features:
  print(train_features[i].shape)
  break

(2048,)


In [14]:
max_length=37

In [15]:

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, LSTM, add, BatchNormalization
from tensorflow.keras.models import Model

# Image feature branch
input_1 = Input(shape=(2048,))
img_dense = Dense(512, activation='relu')(input_1)
img_bn = BatchNormalization()(img_dense)
img_dropout = Dropout(0.5)(img_bn)
img_dense2 = Dense(256, activation='relu')(img_dropout)

# Caption sequence branch
input_2 = Input(shape=(max_length,))
embedding_1 = Embedding(vocab_size, 256, mask_zero=True)(input_2)
cap_dropout = Dropout(0.5)(embedding_1)
cap_lstm = LSTM(256, return_sequences=True)(cap_dropout)
cap_lstm2 = LSTM(256)(cap_lstm)

# Merge image and text
merged = add([img_dense2, cap_lstm2])
merged_bn = BatchNormalization()(merged)
dense_1 = Dense(256, activation='relu')(merged_bn)
dropout_final = Dropout(0.3)(dense_1)
output = Dense(vocab_size, activation='softmax')(dropout_final)

optimizer = Adam(learning_rate=0.0001)
# Final model
caption_model = Model(inputs=[input_1, input_2], outputs=output)
caption_model.compile(loss='categorical_crossentropy', optimizer=optimizer)
caption_model.summary()








Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 37)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │  1,049,088 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 37, 256)   │  2,149,888 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512)       │      2,048 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 37, 256)   │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 37)        │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 37, 256)   │    525,312 │ dropout_1[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │    131,328 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 256)       │    525,312 │ lstm[0][0],       │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256)       │          0 │ dense_1[0][0],    │
│                     │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ add[0][0]         │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │     65,792 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 8398)      │  2,158,286 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,608,078 (25.21 MB)

 Trainable params: 6,606,542 (25.20 MB)

 Non-trainable params: 1,536 (6.00 KB)

In [16]:
caption_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
print(train_features.keys())

dict_keys(['2874876837_80d178ba9b.jpg', '636503038_17ca82b50f.jpg', '3163068926_d28ed3ef53.jpg', '2599131872_65789d86d5.jpg', '2343879696_59a82f496f.jpg', '3542771548_fcb8fa0cba.jpg', '3461677493_5bfb73038e.jpg', '2196050115_e236d91f52.jpg', '2456615908_59cdac6605.jpg', '2089122314_40d5739aef.jpg', '3116731299_6139b25c18.jpg', '3683592946_262e9bfbfd.jpg', '424869823_7aec015d87.jpg', '1104133405_c04a00707f.jpg', '2157173498_2eea42ee38.jpg', '2286823363_7d554ea740.jpg', '2845691057_d4ab89d889.jpg', '842960985_91daf0d6ec.jpg', '3365602213_dd3287a633.jpg', '2259203920_6b93b721ce.jpg', '3242263536_a436f19257.jpg', '2630507245_bea4804288.jpg', '294549892_babb130543.jpg', '2300920203_f29260b1db.jpg', '3042405316_ba3a01926b.jpg', '3295671644_0e10891b6d.jpg', '2127031632_77505e4218.jpg', '484896012_7787d04f41.jpg', '2399114095_c3196ff456.jpg', '241345811_46b5f157d4.jpg', '2394824046_51cec8e5e7.jpg', '3257103624_e76f25ff9e.jpg', '3273489163_8209545810.jpg', '2266142543_b2de18c081.jpg', '46961765

In [17]:
import tensorflow as tf
epochs = 1
no_of_photos = 32
steps = len(train_encoded_captions) // no_of_photos

# Assuming you have a validation generator
#validation_generator = data_generator(val_encoded_captions, validation_features, no_of_photos)

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_encoded_captions, train_features, no_of_photos),
    output_signature=(
        (
            tf.TensorSpec(shape=(None, 2048), dtype=tf.float32),   # X1: image features
            tf.TensorSpec(shape=(None, 37), dtype=tf.int32),       # X2: padded captions
        ),
        tf.TensorSpec(shape=(None, vocab_size), dtype=tf.float32)  # Y: one-hot labels
    )
)
validation_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(val_encoded_captions, validation_features, no_of_photos),
    output_signature=(
        (
            tf.TensorSpec(shape=(None, 2048), dtype=tf.float32),   # X1: image features
            tf.TensorSpec(shape=(None, 37), dtype=tf.int32),       # X2: padded captions
        ),
        tf.TensorSpec(shape=(None, vocab_size), dtype=tf.float32)  # Y: one-hot labels
    )
)

    # Using `fit` with validation data
caption_model.fit(train_dataset,
              epochs=10,
              steps_per_epoch=steps,
              validation_data=validation_dataset,
              validation_steps=len(val_encoded_captions) // no_of_photos,
              verbose=1)


Epoch 1/10
202/227 ━━━━━━━━━━━━━━━━━━━━ 6s 270ms/step - accuracy: 0.1399 - loss: 5.8798

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


227/227 ━━━━━━━━━━━━━━━━━━━━ 63s 242ms/step - accuracy: 0.1451 - loss: 5.7854
Epoch 2/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 82s 361ms/step - accuracy: 0.2868 - loss: 3.9437
Epoch 3/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 56s 247ms/step - accuracy: 0.3340 - loss: 3.4640
Epoch 4/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 82s 361ms/step - accuracy: 0.3545 - loss: 3.2227
Epoch 5/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 82s 360ms/step - accuracy: 0.3685 - loss: 3.0534
Epoch 6/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 55s 240ms/step - accuracy: 0.3783 - loss: 2.9204
Epoch 7/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 55s 243ms/step - accuracy: 0.3874 - loss: 2.8133
Epoch 8/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 54s 238ms/step - accuracy: 0.3958 - loss: 2.7273
Epoch 9/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 55s 241ms/step - accuracy: 0.4016 - loss: 2.6471
Epoch 10/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 54s 240ms/step - accuracy: 0.4107 - loss: 2.5809


In [ ]:
for (img_features, captions), labels in train_dataset.take(1):
    print(f"Image features shape: {img_features.shape}")
    print(f"Captions shape: {captions.shape}")
    print(f"Labels shape: {labels.shape}")

  0%|          | 4/7282 [00:00<02:25, 49.88it/s]

X1 shape: (270, 2048), X2 shape: (270, 37), Y shape: (270, 8398)
Image features shape: (270, 2048)
Captions shape: (270, 37)
Labels shape: (270, 8398)


In [20]:
caption_model.save('model2_name.h5')

In [65]:
def greedy_search(photo):
  photo=photo.reshape(1,2048)
  in_text='<start>'
  for i in range(max_length):
    sequence = [words_to_indices[s] for s in in_text.split(" ") if s in words_to_indices]
    sequence = pad_sequences([sequence], maxlen=max_length, padding='post')
    y_pred = caption_model.predict([photo,sequence],verbose=0)
    y_pred = np.argmax(y_pred[0])
    word = indices_to_words[y_pred]
    in_text += ' ' + word
    if word == '<end>':
      break
  final = in_text.split()
  final = final[1:-1]
  #final = " ".join(final)
  return final

In [ ]:
print(len(test_features))

810


In [20]:
import csv


def generate_caption(model, tokenizer, photo, beam_index=3, max_length=37):
    start = [tokenizer.word_index['start']]

    sequences = [[start, 0.0]]  # (sequence_so_far, cumulative_score)

    while True:
        temp = []
        complete = False  # Track if endseq is generated

        for seq, score in sequences:
            if seq[-1] == tokenizer.word_index['end']:
                # If the sequence already ended, keep it as is
                temp.append([seq, score])
                complete = True
                continue

            sequence_padded = pad_sequences([seq], maxlen=max_length, padding='post')

            if len(photo.shape) == 1:
                photo = np.expand_dims(photo, axis=0)

            preds = model.predict([photo, sequence_padded], verbose=0)
            preds = np.log(preds[0] + 1e-10)  # stability

            top_preds = np.argsort(preds)[-beam_index:]

            for word in top_preds:
                next_seq = seq + [word]
                next_score = score + preds[word]
                temp.append([next_seq, next_score])

        sequences = sorted(temp, reverse=False, key=lambda tup: tup[1])
        sequences = sequences[-beam_index:]

        # stopping condition
        if complete or len(sequences[0][0]) >= max_length:
            break

    final_seq = sequences[-1][0]
    final_caption = [tokenizer.index_word.get(idx, '') for idx in final_seq]

    # clean the caption
    final_caption = final_caption[1:]  # remove start token
    if 'end' in final_caption:
        end_index = final_caption.index('end')
        final_caption = final_caption[:end_index]

    return ' '.join(final_caption)



# Load the tokenizer and generate captions for images
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# for img_id in test_features:
#     photo = test_features[img_id]
#     caption = generate_caption(caption_model, tokenizer, photo)
#     print(f"{img_id}: {caption}")

# Open the CSV file in write mode (creating it if it doesn't exist)
with open('generated_captions.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'Caption'])  # Write header row

    for img_id in test_features:  # Assuming test_features is a dictionary of image IDs and features
        photo = test_features[img_id]
        caption =generate_caption(caption_model, tokenizer, photo, beam_index=5, max_length=37)

        # Print the generated caption to the console
        print(f"Image ID: {img_id}.jpg - Caption: {caption}")

        # Write each generated caption to the CSV file as it is generated
        writer.writerow([f"{img_id}.jpg", caption])


Image ID: 544257613_d9a1fea3f7.jpg.jpg - Caption: a closeup of a closeup of a woman
Image ID: 2518508760_68d8df7365.jpg.jpg - Caption: a brown dog is running on a track
Image ID: 2133650765_fc6e5f295e.jpg.jpg - Caption: a boy jumps into a pool
Image ID: 2914737181_0c8e052da8.jpg.jpg - Caption: a man in a wetsuit is surfing in the ocean
Image ID: 3203453897_6317aac6ff.jpg.jpg - Caption: a group of people are walking down a city street
Image ID: 3708172446_4034ddc5f6.jpg.jpg - Caption: a man is sitting on a bench in a park
Image ID: 1775029934_e1e96038a8.jpg.jpg - Caption: a brown dog is running through the grass
Image ID: 3225226381_9fe306fb9e.jpg.jpg - Caption: two dogs play in the grass
Image ID: 3415113018_7b95ddcd11.jpg.jpg - Caption: two men play soccer
Image ID: 2480820830_bdec1f5b76.jpg.jpg - Caption: a black dog is running through the surf
Image ID: 3307563498_e2b4f19272.jpg.jpg - Caption: a group of people are standing on the beach
Image ID: 3215896272_e9b4b547a9.jpg.jpg - Capt

In [22]:
import csv

# Open the original CSV file to read and a new file to write the fixed data
with open('/content/generated_captions.csv', 'r', newline='') as infile, open('fixed_generated_captions5.csv', 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Read the header
    header = next(reader)
    writer.writerow(header)  # Write the header to the new file

    # Iterate over each row
    for row in reader:
        img_name = row[0]
        caption = row[1]

        # Fix the image name by removing extra '.jpg' if it appears at the end
        if img_name.endswith('.jpg.jpg'):
            img_name = img_name[:-4]  # Remove the extra '.jpg' part

        # Write the fixed row to the new CSV
        writer.writerow([img_name, caption])

# Inform user that the fix is done
print("CSV file has been fixed and saved as 'fixed_generated_captions2.csv'")


CSV file has been fixed and saved as 'fixed_generated_captions2.csv'
